## Experiments

Pipeline Steps:

*from Driffta*

- Given region and tile
    - load Experiment.json, channelNames.txt, and processingOptions.json
    - for each channel + z
        - construct input image name
        - if not keyence, copy the file first before opening?
        - if channel_arrangement is "color" do max z-projection
        - \* get explanation on what to do with H&E
        - aggregate images into stack
    - choose best focus z-plane for each cycle
        - currently, the same plane is used for all cycles though
    - run drift compensation
    - crop off tile overlap 
        - note that this is different when "useBleachMinimizingCrop" is set in processingOptions
    - run deconvolution
    - set LUTS (for either 3 or 4 channels)
    - do background subtraction if set by experiment.bgSub
    - save result
    - also save single z-stack using best focus plans in separate dir
    


In [1]:
from codex import config

In [1]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=0, processes=True)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:53473,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
??LocalCluster

In [11]:
import dask.array as da

def fn(a):
    import time
    import numpy as np
    for i in range(100):
        np.matmul(np.ones((1000, 1000)), np.ones((1000, 1000)))
    return a*2
res = client.map(fn, [1, 2, 3])
res

[<Future: status: pending, key: fn-50ac6fd4db73382b2d803f919be8a238>,
 <Future: status: pending, key: fn-b98761c5920ab097086c24e37aaf27e8>,
 <Future: status: pending, key: fn-370c033fb5625485f704265759103b71>]

In [19]:
[r.result() for r in res]

[2, 4, 6]

In [21]:
cluster.close()

In [7]:
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
fd_data.load_celegans_channel('CY3').shape()

{'actual': None, 'data': (104, 712, 672), 'kernel': (104, 712, 672)}

In [19]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10807682206669945409]

In [1]:
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
import numpy as np
import logging
logger = logging.getLogger('NB')

def run_mult(device):
    for i in range(1000):
        np.matmul(np.ones((1000, 1000)), np.ones((1000, 1000)))
    return 1

def run_deconvolution(device):
    import tensorflow as tf
    import os
    logger.error('Device set: {}'.format(os.getenv('CUDA_VISIBLE_DEVICES')))
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True
    acq = fd_data.load_bars()
    algo = fd_restoration.RichardsonLucyDeconvolver(3).initialize()
    res = algo.run(acq, niter=1000, session_config=config).data
    return res

def set_device(device):
    import os
    logger.error('Setting device to {}'.format(device))
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"]= device
    
def init_logging():
    import logging
    

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=2, threads_per_worker=1,
                       processes=True, memory_limit=16e9)
client = Client(cluster)

try:
    workers = cluster.workers
    client.run(init_logging)
    client.run(set_device, '0', workers=[workers[0].address])
    client.run(set_device, '1', workers=[workers[1].address])
    print('Done set device')
    res = client.map(run_deconvolution, ['/gpu:0', '/gpu:1'])
    #res = client.map(run_mult, ['/cpu:0']*3)
    res = [r.result() for r in res]
    logs = client.get_worker_logs()
finally:
    client.close()
    cluster.close()

Setting device to 0
Setting device to 1


Done set device


KeyboardInterrupt: 

In [40]:
logs[list(logs.keys())[0]]

[['INFO',
  'distributed.worker - INFO -       Start worker at:      tcp://127.0.0.1:52195'],
 ['INFO',
  'distributed.worker - INFO -          Listening to:      tcp://127.0.0.1:52195'],
 ['INFO',
  'distributed.worker - INFO -              nanny at:            127.0.0.1:52188'],
 ['INFO',
  'distributed.worker - INFO - Waiting to connect to:      tcp://127.0.0.1:52186'],
 ['INFO',
  'distributed.worker - INFO - -------------------------------------------------'],
 ['INFO',
  'distributed.worker - INFO -               Threads:                          1'],
 ['INFO',
  'distributed.worker - INFO -                Memory:                   16.00 GB'],
 ['INFO',
  'distributed.worker - INFO -       Local Directory: C:\\Users\\User\\repos\\codex\\python\\notebooks\\experiments\\dask-worker-space\\worker-v3f83e63'],
 ['INFO',
  'distributed.worker - INFO - -------------------------------------------------'],
 ['INFO',
  'distributed.worker - INFO -         Registered to:      tcp://127.0.0.